In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 7.1 MB/s 
     |████████████████████████████████| 596 kB 56.2 MB/s 
     |████████████████████████████████| 6.6 MB 7.7 MB/s 
     |████████████████████████████████| 101 kB 12.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import pandas as pd
import numpy as np
import random
import json
import time
import datetime
import os

from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig

import torch
from transformers import BertTokenizer
from transformers import get_linear_schedule_with_warmup

In [3]:
from google.colab import drive
drive.mount("/drive")

Mounted at /drive


In [4]:
os.chdir("/drive/MyDrive/bert_herb_drug/")

In [5]:
!pwd

/drive/MyDrive/bert_herb_drug


In [6]:
model_out = "./output_new"

In [8]:
data_fname = 'datasets/train_new.csv'
df = pd.read_csv(data_fname)
df.head()

,Unnamed: 0,sentences,entity_1_mention,entity_2_mention,type,kfold
0,0,interactions of several commonly used herbal m...,milk thistle,indinavir,POSITIVE,0
1,1,"0 years, and their most-used medications were ...",mint,atorvastatin,SPECULATIVE,0
2,2,<e1> khat </e1> chewing has been shown to re...,khat,amoxicillin,POSITIVE,0
3,3,conclusion: the growth inhibitory activity of...,t. bellerica,doxorubicin,POSITIVE,0
4,4,by taking advantage of the high protein bindin...,chan su,digoxin,POSITIVE,0


In [9]:
df['type'].value_counts()

POSITIVE       245
SPECULATIVE    102
NEGATIVE        27
Name: type, dtype: int64

In [10]:
label_map_dic = {
    "POSITIVE": 0,
    "SPECULATIVE": 1,
    "NEGATIVE": 2,
}
df['label'] = df['type'].map(label_map_dic)
df.head()

,Unnamed: 0,sentences,entity_1_mention,entity_2_mention,type,kfold,label
0,0,interactions of several commonly used herbal m...,milk thistle,indinavir,POSITIVE,0,0
1,1,"0 years, and their most-used medications were ...",mint,atorvastatin,SPECULATIVE,0,1
2,2,<e1> khat </e1> chewing has been shown to re...,khat,amoxicillin,POSITIVE,0,0
3,3,conclusion: the growth inhibitory activity of...,t. bellerica,doxorubicin,POSITIVE,0,0
4,4,by taking advantage of the high protein bindin...,chan su,digoxin,POSITIVE,0,0


In [11]:
# Instantiate the Bert tokenizer
# sic-ber  biobert 
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [12]:
def get_inputs(df):
    input_ids = []
    attention_masks = []
    labels = []

    MAX_LENGTH = 128

    # For every sentence...
    for i,row in df.iterrows():
        #print(row['entity1'])
        sent = row['sentences'] + str('[SEP]') + row['entity_1_mention'] + str('[SEP]') + row['entity_2_mention'] 
        labels.append(int(row['label']))
        encoded_dict = tokenizer.encode_plus(
                          sent,                          # Sentence to encode.
                          add_special_tokens = True,     # Add '[CLS]' and '[SEP]'
                          max_length = MAX_LENGTH,       # Pad & truncate all sentences.
                          pad_to_max_length = True,
                          return_attention_mask = True,  # Construct attn. masks.
                          return_tensors = 'pt',         # Return pytorch tensors.
                      )

        # Add the encoded sentence to the list.    
        input_ids.append(encoded_dict['input_ids'])

        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])

    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)
    
    return input_ids,attention_masks,labels

In [13]:
kfold = 0
train_df = df[df.kfold != kfold].reset_index(drop=True)
valid_df = df[df.kfold == kfold].reset_index(drop=True)  

train_input_ids, train_attention_masks, train_labels = get_inputs(train_df)
valid_input_ids, valid_attention_masks, valid_labels = get_inputs(valid_df)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [14]:
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
val_dataset = TensorDataset(valid_input_ids, valid_attention_masks, valid_labels)

In [15]:

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [16]:
model = BertForSequenceClassification.from_pretrained(
               'bert-base-uncased', 
               num_labels = len(label_map_dic), # The number of output labels. 2 for binary classification.  
               output_attentions = False,
               output_hidden_states = False)   

# Tell pytorch to run this model on the GPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# send model to device
model.to(device);

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [17]:
optimizer = AdamW(model.parameters(),
                  lr = 5e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 2,I have already seen that the model starts overfitting beyound 2 epochs
epochs = 5

# Total number of training steps is [number of batches] x [number of epochs]. 
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                       num_warmup_steps = 0, # Default value in run_glue.py
                       num_training_steps = total_steps)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [18]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [19]:
# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# store a number of quantities such as training and validation loss,validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):

    # training
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode.
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader andcopy each tensor to the GPU
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing abackward pass.
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        outputs = model(b_input_ids, 
                token_type_ids=None, 
                attention_mask=b_input_mask, 
                labels=b_labels)

        # Accumulate the training loss over all of the batches
        loss= outputs[0]
        logits = outputs[1]  
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.4f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
    
    # ========================================
    # Save model of every epoch
    output_dir = model_out + '/models/'
    # Create output directory if needed
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    models_dir = output_dir + 'bert_model_epoch' + str(epoch_i + 1)
    # Create output directory if needed
    if not os.path.exists(models_dir):
        os.makedirs(models_dir)

    print("Saving model to %s" % models_dir)

    # Save a trained model, configuration and tokenizer using `save_pretrained()`.
    # They can then be reloaded using `from_pretrained()`
    model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
    model_to_save.save_pretrained(models_dir)
    tokenizer.save_pretrained(models_dir)
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0.0
    total_eval_loss = 0.0
    nb_eval_steps = 0.0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        with torch.no_grad():        
            outputs= model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask,
                    labels=b_labels)
            
        # Accumulate the validation loss.
        loss= outputs[0]
        logits = outputs[1] 
        
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.4f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.4f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

======== Epoch 1 / 5 ========
Training...

  Average training loss: 0.8643
  Training epcoh took: 0:00:08
Saving model to ./output_new/models/bert_model_epoch1

Running Validation...
  Accuracy: 0.6932
  Validation Loss: 0.8141
  Validation took: 0:00:00
======== Epoch 2 / 5 ========
Training...

  Average training loss: 0.6734
  Training epcoh took: 0:00:06
Saving model to ./output_new/models/bert_model_epoch2

Running Validation...
  Accuracy: 0.8059
  Validation Loss: 0.6678
  Validation took: 0:00:01
======== Epoch 3 / 5 ========
Training...

  Average training loss: 0.4784
  Training epcoh took: 0:00:06
Saving model to ./output_new/models/bert_model_epoch3

Running Validation...
  Accuracy: 0.8059
  Validation Loss: 0.5443
  Validation took: 0:00:01
======== Epoch 4 / 5 ========
Training...

  Average training loss: 0.3151
  Training epcoh took: 0:00:06
Saving model to ./output_new/models/bert_model_epoch4

Running Validation...
  Accuracy: 0.7955
  Validation Loss: 0.4475
  Valid

In [20]:
import os
# Display floats with two decimal places.
pd.set_option('precision', 4)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

output_dir = model_out + '/models/'
# # Create output directory if needed
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

df_stats.to_csv(output_dir + 'df_train_stats.csv',index=None)

# according df_stats, pick the best performace model
df_stats

,Training Loss,Valid. Loss,Valid. Accur.,Training Time,Validation Time
epoch,,,,,
1,0.8643,0.8141,0.6932,0:00:08,0:00:00
2,0.6734,0.6678,0.8059,0:00:06,0:00:01
3,0.4784,0.5443,0.8059,0:00:06,0:00:01
4,0.3151,0.4475,0.7955,0:00:06,0:00:01
5,0.2284,0.3700,0.8258,0:00:06,0:00:01
